In [1]:
from IPython.display import Image

### PPO 的四个module

- actor, critic, reference, reward
    - 早期的 trl ppov1 实现，colocate actor/critic
        - critic a mlp head;

In [3]:
Image(url='./imgs/openrlhf_ppo.png', width=500)

In [4]:
Image(url='./imgs/openrlhf_flow.png', width=500)

### 代码细节

- 训练入口
    - train_ppo_ray.py
- 在main中我们启动了driver进程，并执行训练函数train(args)，这里主要做了如下几件事：
    - 在 ray 集群上部署 Actor/Ref/Critic/RM 实例
    - 在 ray 集群上部署 vllm_engines 实例
    - 配置 Actor 和 vllm_engines 之间的通讯，用于传递权重
    - 训练 Actor 和 Critic 模型
- 谈到“Actor”时，会使用Ray-Actor和PPO-Actor来做区分，从之前的介绍中可知，Ray-Actor是指部署在Ray集群中的远端class，PPO-Actor/Ref/Critic/RM都属于Ray-Actor。
- colocate_actor_ref
    - 是否共同部署；
    - `num_gpus_per_actor=0.75 if pg else 1,`
    - 当我们在pg（placement group）指向的预留资源中分配模型实例时，再进一步指定每个实例占据一张gpu的多少部分
        - 等于1说明每个实例占满一张gpu，即“非共同部署”
        - 小于1说明每个实例只占部分gpu，即“共同部署”，例如PPO-Actor/Ref共同部署在一张卡上

```
# NOTE(wuxibin): Why don't we allocate 0.5 gpu for each actor when colocate models?
# Say we have 1 node with 4 GPUs, and num_gpus_per_node for each model is 4.
# If we allocate 0.5 gpu for both actor and ref model, then gpu allocation is
#   |actor|actor|actor|actor|  ref | ref  | ref  | ref |
#   |GPU0 |GPU0 |GPU1 |GPU1 | GPU2 | GPU2 | GPU3 | GPU3 |
#
# So 0.75/0.25 gpu is a tricky to let Ray spread all models evenly on all gpus.
#   |actor| ref  |actor| ref  |actor| ref  |actor|ref  |
#   |GPU0 | GPU0 |GPU1 | GPU1 |GPU2 | GPU2 |GPU3 | GPU3 |
```

In [3]:
Image(url='https://pic3.zhimg.com/v2-a7445701e230850618a1a055ad9a8cec_r.jpg', width=600)

- PPORayActorGroup中维护着self._actor_handlers，它是一个List[ray.actor.ActorHandle]，列表中每个元素表示某个远端Ray-Actor的引用，而这个远端Ray-Actor可以是PPO-Actor/Ref/Critic/RM实例。如前文所说，我们可以在ray集群中的任何位置调用这个handler，来对相应的远端Ray-Actor执行操作。

### scripts

- openrlhf.cli.train_ppo_ray
    - actor_num_gpus_per_node
    - ref_num_gpus_per_node
    - critic_num_gpus_per_node（经典 ppo，GRPO/RLOO 不再需要）
    - reward_num_gpus_per_node（经典 ppo，GRPO/RLOO 不再需要）
    - vllm_num_engines